-----

# !!!!   Zwróćcie uwagę accuracy naszego modelu wzrosło! :)  !!!!!

tensor(0.9211)

vs.

tensor(0.9474)

To dzięki              

self.loss = nn.MSELoss(reduction='sum') 

vs.   

self.loss = nn.CrossEntropyLoss()

Zwróćcie też uwagę na to:

predictions = model(input_test_tensors)
predicted_labels = torch.argmax(predictions, dim=1)
torch.sum(torch.eq(torch.tensor(label_test), predicted_labels)) / len(predicted_labels)

W przeciwieństwie do poprzedniej sieci, gdzie wykonaliśmy na ostatniej warstwie regresję i przewidywaliśmy
wartości, tutaj mamy do czynienia z klasyfikacją i mamy skuteczność (accuracy) klasyfikacji jako jedną z możliwych
miar: 92% vs 95%.
W celu wykonania takiej klasyfikacji na ostatniej warstwie musimy wykorzystać funkcję argmax, czyli
popatrzcie dostajemy np. w klasyfikacji 3 wartości np. [0.4,0.8,0.1], po wybraniu argmax zwraca  nam 1, czyli wybieramy gatunek o indeksie 1 zgodnie z naszą faktoryzacją i oznaczeniami one_hot.



In [57]:
!pip install lightning
import torch # torch will allow us to create tensors.
import torch.nn as nn # torch.nn allows us to create a neural network.
import torch.nn.functional as F # nn.functional give us access to the activation and loss functions.
from torch.optim import Adam # optim contains many optimizers. This time we're using Adam

import lightning as L # lightning has tons of cool tools that make neural networks easier
from torch.utils.data import TensorDataset, DataLoader # these are needed for the training data

import pandas as pd # We'll use pandas to read in the data and normalize it
from sklearn.model_selection import train_test_split
url = "./iris.txt"
df = pd.read_table(url, sep=",", header=None)
df.head()
df.columns = ["sepal_length",
              "sepal_width",
              "petal_length",
              "petal_width",
              "class"]
df.head()
df.shape
df['class'].nunique()
df['class'].unique()
for class_name in df['class'].unique(): # for each unique class name...
    ## ...print out the number of rows associated with it
    print(class_name, ": ", sum(df['class'] == class_name), sep="")
df[['petal_width', 'sepal_width']].head()
input_values = df[['petal_width', 'sepal_width']]
input_values.head()
label_values = df['class']
label_values.head()
classes_as_numbers = label_values.factorize()[0]
classes_as_numbers 
input_train, input_test, label_train, label_test = train_test_split(input_values,
                                                                    classes_as_numbers,
                                                                    test_size=0.25,
                                                                    stratify=classes_as_numbers,
                                                                    random_state=42)
one_hot_label_train = F.one_hot(torch.tensor(label_train)).type(torch.float32)
max_vals_in_input_train = input_train.max()
max_vals_in_input_train
min_vals_in_input_train = input_train.min()

min_vals_in_input_train
input_train = (input_train - min_vals_in_input_train) / (max_vals_in_input_train - min_vals_in_input_train)
input_train.head()
input_test = (input_test - min_vals_in_input_train) / (max_vals_in_input_train - min_vals_in_input_train)
input_test.head()
input_train_tensors = torch.tensor(input_train.values).type(torch.float32)
input_test_tensors = torch.tensor(input_test.values).type(torch.float32)

train_dataset = TensorDataset(input_train_tensors, one_hot_label_train)
train_dataloader = DataLoader(train_dataset)

Iris-setosa: 50
Iris-versicolor: 50
Iris-virginica: 50


----

<a id="train"></a>
# Training our Neural Network

In [58]:
class MultipleInsOuts(L.LightningModule):

    def __init__(self):

        super().__init__()
        L.seed_everything(seed=42)
        self.input_to_hidden = nn.Linear(in_features=2, out_features=2, bias=True)
        self.hidden_to_output = nn.Linear(in_features=2, out_features=3, bias=True)

        self.loss = nn.MSELoss(reduction='sum')


    def forward(self, input):
        ## First, we run the input values to the activation functions
        ## in the hidden layer
        hidden = self.input_to_hidden(input)
        output_values = self.hidden_to_output(torch.relu(hidden))

        return(output_values)


    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.001)


    def training_step(self, batch, batch_idx):
        inputs, labels = batch

        outputs = self.forward(inputs)

        loss = self.loss(outputs, labels)

        return loss

Training our new neural network means we create a model from the new class, `MultipleInsOuts`...

In [59]:
model = MultipleInsOuts()

Seed set to 42


In [60]:
trainer = L.Trainer(max_epochs=10)
trainer.fit(model, train_dataloaders=train_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type    | Params | Mode 
-----------------------------------------------------
0 | input_to_hidden  | Linear  | 6      | train
1 | hidden_to_output | Linear  | 9      | train
2 | loss             | MSELoss | 0      | train
-----------------------------------------------------
15        Trainable params
0         Non-trainable params
15        Total params
0.000     Total estimated model params size (MB)
/home/piotr/anaconda3/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=10` reached.


In [61]:
predictions = model(input_test_tensors)

Now, because our neural network has three outputs, one for **Setosa**, one for **Versicolor**, and one for **Virginica**, we should get 3 values for each row in `input_test_tensors`. We can verify that by looking at the first few rows of `predictions`.

In [62]:
predictions[0:4,]

tensor([[0.7585, 0.0282, 0.0331],
        [0.1994, 0.3877, 0.4948],
        [0.2486, 0.3490, 0.3868],
        [0.1941, 0.3776, 0.3712]], grad_fn=<SliceBackward0>)

We can determine which species was predicted in `predictions` by selecting the index in each row that corresponding to the largest value, and we do that with `torch.argmax()`. `torch.argmax()` returns a tensor that contains the indices with the largest values for each row.

In [63]:
predicted_labels = torch.argmax(predictions, dim=1) ## dim=0 applies argmax to rows, dim=1 applies argmax to columns
predicted_labels[0:4]

tensor([0, 2, 2, 1])

In the first row index 0 had the largest value. Thus, the first prediction corresponds to **Setosa**. The second, third, and fourth rows predicted 2, which corresponds to **Virginica**.

Now, let's compare what the neural network predicted in `predicted_labels` to the known values in `label_test` and calculate the percentage of correct predictions. We do this by adding up the number of times an element in `predicted_labels` equals the corresponding element in `label_test` and dividing by the number of elements in `predicted_labels`.

In [64]:
## Now compare predicted_labels with test_labels to calculate accuracy
## NOTE: torch.eq() computes element-wise equality between two tensors.
torch.sum(torch.eq(torch.tensor(label_test), predicted_labels)) / len(predicted_labels)

tensor(0.7368)

And we see that our neural network only correctly predicts 74% of the testing data. This isn't very good. So, will training our model for more epochs improve the model's predictions?

One way to answer that question is to just train for longer and see what happens.

To add additional epochs to the training, we first identify where the checkpoint file is with the following command.

In [65]:
path_to_checkpoint = trainer.checkpoint_callback.best_model_path ## By default, "best" = "most recent"

In [66]:
## First, create a new Lightning Trainer
trainer = L.Trainer(max_epochs=100) # Before, max_epochs=10, so, by setting it to 100, we're adding 90 more.

## Then call trainer.fit() using the path to the most recent checkpoint files
## so that we can pick up where we left off.
trainer.fit(model, train_dataloaders=train_dataloader, ckpt_path=path_to_checkpoint)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at /home/piotr/Documents/zajecia/DNN/CS-DNN/tutorials/3/lightning_logs/version_11/checkpoints/epoch=9-step=1120.ckpt
/home/piotr/anaconda3/lib/python3.8/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.a

Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=100` reached.


# ACCURACY 1

Now, let's run the testing data through the network and calculate the accuracy. We'll do this just like we did before.

## ARGMAX - pamiętajcie - klasyfikacja!

In [67]:
predictions = model(input_test_tensors)
predicted_labels = torch.argmax(predictions, dim=1)
torch.sum(torch.eq(torch.tensor(label_test), predicted_labels)) / len(predicted_labels)

tensor(0.9211)

After 100 training epochs, we correctly classified 92% of the testing data. This means adding more training was helpful!

<a id="predict"></a>
# Make a Prediction with New Data

In [68]:
normalized_values = ([0.2, 3.0] - min_vals_in_input_train) / (max_vals_in_input_train - min_vals_in_input_train)
normalized_values

petal_width    0.041667
sepal_width    0.416667
dtype: float64

Then we convert `normalized_values` into a tensor and pass it to the model to see what it predicts.

In [69]:
model(torch.tensor(normalized_values).type(torch.float32))

tensor([ 0.7956,  0.1990, -0.0419], grad_fn=<ViewBackward0>)

And first output has the largest value, meaning that the neural network predicts that the measurements come from **Setosa**.

# We now increase the accuracy through Cross Entropy.

In [71]:
class MultipleInsOuts(L.LightningModule):

    def __init__(self):

        super().__init__()
        L.seed_everything(seed=42)
        self.input_to_hidden = nn.Linear(in_features=2, out_features=2, bias=True)
        self.hidden_to_output = nn.Linear(in_features=2, out_features=3, bias=True)
        self.loss = nn.CrossEntropyLoss()


    def forward(self, input):
        hidden = self.input_to_hidden(input)
        output_values = self.hidden_to_output(torch.relu(hidden))

        return(output_values)


    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.001)


    def training_step(self, batch, batch_idx):
        inputs, labels = batch

        outputs = self.forward(inputs)

        loss = self.loss(outputs, labels)

        return loss

In [72]:
model = MultipleInsOuts()

Seed set to 42


In [73]:
trainer = L.Trainer(max_epochs=10)
trainer.fit(model, train_dataloaders=train_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params | Mode 
--------------------------------------------------------------
0 | input_to_hidden  | Linear           | 6      | train
1 | hidden_to_output | Linear           | 9      | train
2 | loss             | CrossEntropyLoss | 0      | train
--------------------------------------------------------------
15        Trainable params
0         Non-trainable params
15        Total params
0.000     Total estimated model params size (MB)


Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=10` reached.


In [74]:
predictions = model(input_test_tensors)

Now, because our neural network has three outputs, one for **Setosa**, one for **Versicolor**, and one for **Virginica**, we should get 3 values for each row in `input_test_tensors`. We can verify that by looking at the first few rows of `predictions`.

In [75]:
predictions[0:4,]

tensor([[ 0.9644, -0.1377,  0.0748],
        [ 0.0507,  0.5664,  0.8702],
        [ 0.1509,  0.4944,  0.6980],
        [ 0.1030,  0.5353,  0.6764]], grad_fn=<SliceBackward0>)

We can determine which species was predicted in `predictions` by selecting the index in each row that corresponding to the largest value, and we do that with `torch.argmax()`. `torch.argmax()` returns a tensor that contains the indices with the largest values for each row.

In [76]:
## Select the output with highest value...
predicted_labels = torch.argmax(predictions, dim=1) ## dim=0 applies argmax to rows, dim=1 applies argmax to columns
predicted_labels[0:4] # print out the first 4 predictions

tensor([0, 2, 2, 2])

In the first row index 0 had the largest value. Thus, the first prediction corresponds to **Setosa**. The second, third, and fourth rows predicted 2, which corresponds to **Virginica**.

Now, let's compare what the neural network predicted in `predicted_labels` to the known values in `label_test` and calculate the percentage of correct predictions. We do this by adding up the number of times an element in `predicted_labels` equals the corresponding element in `label_test` and dividing by the number of elements in `predicted_labels`.

In [77]:
torch.sum(torch.eq(torch.tensor(label_test), predicted_labels)) / len(predicted_labels)

tensor(0.6579)

And we see that our neural network only correctly predicts 66% of the testing data. This isn't very good. So, will training our model for more epochs improve the model's predictions?

One way to answer that question is to just train for longer and see what happens.

To add additional epochs to the training, we first identify where the checkpoint file is with the following command.

In [78]:
path_to_checkpoint = trainer.checkpoint_callback.best_model_path ## By default, "best" = "most recent"

Then we create a new Lightning Trainer, just like before, but we set the number of epochs to 100. Given that we already trained for 10 epochs, this means we'll do 90 more.

In [79]:
trainer = L.Trainer(max_epochs=100)
trainer.fit(model, train_dataloaders=train_dataloader, ckpt_path=path_to_checkpoint)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at /home/piotr/Documents/zajecia/DNN/CS-DNN/tutorials/3/lightning_logs/version_13/checkpoints/epoch=9-step=1120.ckpt
/home/piotr/anaconda3/lib/python3.8/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:360: The dirpath has changed from '/home/piotr/Documents/zajecia/DNN/CS-DNN/tutorials/3/lightning_logs/version_13/checkpoints' to '/home/piotr/Documents/zajecia/DNN/CS-DNN/tutorials/3/lightning_logs/version_14/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params | Mode 
--------------------------------------------------------------
0 | input_to_hidden  | Linear           | 6      | train
1 | hidden_to_outpu

Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=100` reached.


# ACCURACY 2

## ARGMAX - pamiętajcie - klasyfikacja!

Now, let's run the testing data through the network and calculate the accuracy. We'll do this just like we did before.

In [80]:
predictions = model(input_test_tensors)
predicted_labels = torch.argmax(predictions, dim=1)
torch.sum(torch.eq(torch.tensor(label_test), predicted_labels)) / len(predicted_labels)

tensor(0.9474)

# !!!!   Zwróćcie uwagę accuracy naszego modelu wzrosło! :)  !!!!!

tensor(0.9211)

vs.

tensor(0.9474)

To dzięki         self.loss = nn.CrossEntropyLoss()

### After 100 training epochs, we correctly classified 95% of the testing data. This means adding more training was helpful!

<a id="predict"></a>
# Make a Prediction with New Data

In [81]:
normalized_values = ([0.2, 3.0] - min_vals_in_input_train) / (max_vals_in_input_train - min_vals_in_input_train)
normalized_values

petal_width    0.041667
sepal_width    0.416667
dtype: float64

Then we convert `normalized_values` into a tensor and pass it to the model to see what it predicts.

In [82]:
model(torch.tensor(normalized_values).type(torch.float32))

tensor([ 6.0464,  1.5474, -6.4604], grad_fn=<ViewBackward0>)

And first output has the largest value, meaning that the neural network predicts that the measurements come from **Setosa**.